<a href="https://colab.research.google.com/github/Yongshilin/Simple-Calculator/blob/main/Interactive_DRIs_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a simple Dietary Reference Intakes (DRIs) Calculator that user can interact with.
User can choose diffrent buttons from the main menu, including:
- A)dd to an existing file
- L)oad a file and view data
- P)rint current data
- R)eset and add new data
- S)ave current list to a file
- Q)uit.

The user can add more than one person's infomation, display the result as a whole, and save the result into a local file. The user can load the saved file, add new infomration or reset & add new data.

The original data source is from https://ods.od.nih.gov/HealthInformation/nutrientrecommendations.aspx.

This project is only for self-learning.

In [1]:
import pandas as pd
import json
import re
import math

In [2]:
mainmenu = "\nMain Menu:\nA)dd to an existing file \nL)oad a file and view data \nP)rint current data \nR)esset and add new data \nS)ave current list to a file\nQ)uit:\n"
question = "Enter your name to start or q to quit"
nofile = "\nThe file doesn't exists\n"
nocalc = "\nThere are no calculations to display\nLoad a file or perform new calculations\n"

In [3]:
def validate(vString):
    while True:
        try:
            if int(float(vString)) >= 0:
                vString = int(float(vString))
                break
            else:
                vString = input("Negative values are not accepted. Please provide a positive value:\n")
        except ValueError:
            vString = input("Non-numeric values are not accepted. Please provide a positive numeric value:\n")
    return vString

In [4]:
def getWeightHeight():
    weight = float(validate(input("Enter your weight (in kilograms): ")))
    height = float(validate(input("Enter your height (in centimeter): ")))
    return weight, height

In [5]:
def calcBMI(weight, height):
    bmi = weight / ((height/100) ** 2)
    if bmi < 18.5:
        bmi_cat = "Underweight"
    elif 18.5 <= bmi < 24.9:
        bmi_cat = "Normal Weight"
    elif 25 <= bmi < 29.9:
        bmi_cat = "Overweight"
    else:
        bmi_cat = "Obesity"
    return bmi, bmi_cat

In [6]:
def printResults(NutritionCalList, nutri_data):

    # Print the table headers first.
    print('*********\n\nThe Results of BMI Calculations\n\n********')
    print("{0:<16}{1:<10}{2:<10}{3:<10}{4:<16}{5:<14}{6:<14}{7:<14}{8:<14}".format("Name", "Weight", "Height", "BMI", "BMI Category","Age(year)",  "Gender", "Pregnancy", "Lactation"))

    for row in NutritionCalList:
        name = row[0] if row[0] is not None else "N/A"
        weight = row[1] if row[1] is not None else 0.0
        height = row[2] if row[2] is not None else 0.0
        bmi = row[3] if row[3] is not None else 0.0
        bmi_cat = row[4] if row[4] is not None else "N/A"
        age = row[5] if row[5] is not None else "N/A"
        gender = row[6] if row[6] is not None else "N/A"
        pregnancy = row[7] if row[7] is not None else "N/A"
        lactation = row[8] if row[8] is not None else "N/A"

        print("{0:<16}{1:<10.2f}{2:<10.2f}{3:<10.2f}{4:<16}{5:<14}{6:<14}{7:<14}{8:<14}".format(
            name, weight, height, bmi, bmi_cat, age, gender, pregnancy, lactation))

    print("\n")
    # Print the suggested average daily nutrition amounts for each person
    for row in NutritionCalList:
        name = row[0]
        nutri_data = row[9]  # This is the list of dictionaries
        print(f"{name}, Your suggested average daily nutrition amounts are:\n")
        if isinstance(nutri_data, list) and nutri_data:
            for record in nutri_data:
                for key, value in record.items():
                    if key not in ['Gender', 'Pregnancy', 'Lactation', 'Age(year)']:  # Exclude specific keys
                        if isinstance(value, float) and math.isnan(value):
                            value = 'null'
                        print(f"{key}: {value}")
                print("\n")
        else:
            print("No nutrition data available.\n")

    print('*********\n\n')

In [7]:
def askData(df, NutritionCalList):    #need to pass NutritionCalList as a parameter to the askData function, because NutritionCalList is defined only within the main function, and it's not accessible within the askData function.
    print(question)
    name = input().strip().capitalize()

    while (name.lower() !="q"):
        weight, height = getWeightHeight()
        bmi,bmi_cat = calcBMI(weight, height)

        # Strip whitespace from 'Gender' column
        df['Gender'] = df['Gender'].str.strip()

        # Clean column names to remove any newline characters and extra spaces (for later to print the suggested result for the user and make sure the column name is in one line)
        df.columns = [col.replace('\n', ' ').strip() for col in df.columns]

        #ask additional info
        user_age = float(input("Enter your age(in year)"))
        gender = input("Enter your gender (M for male/F for female): ").upper()
        while gender not in ['M', 'F']:
            print("Invalid gender. Please enter M for male or F for female.")
            gender = input("Enter your gender (M for male/F for female): ").upper()

        pregnancy = lactation = None  # Initialize pregnancy and lactation variables

        if gender == 'F':
            while True:
                pregnancy = input("Are you pregnant (Y/N): ").upper()
                if pregnancy in ['Y', 'N']:
                    break
                else:
                    print("Invalid input. Please enter Y for Yes or N for No.")

            while True:
                lactation = input("Are you lactating (Y/N): ").upper()
                if lactation in ['Y', 'N']:
                    break
                else:
                    print("Invalid input. Please enter Y for Yes or N for No.")


        # Filter the DataFrame based on gender, age, pregnancy, and lactation
        filtered_df = df[
                (df['Gender'] == ('Females' if gender == 'F' else 'Males'))
            &
            (
                (df['Age(year)'].apply(lambda x: float(x.split('-')[0])) <= user_age) &  #left exclusive, in case 2 row matched
                (df['Age(year)'].apply(lambda x: float(x.split('-')[1].split()[0])) >= user_age)
                )
        ]

        # Further filter based on pregnancy and lactation status
        if gender == 'F':
            filtered_df = filtered_df[
                (filtered_df['Pregnancy'] == pregnancy) |
                (filtered_df['Pregnancy'].isna() & (pregnancy == 'N'))
            ]
            filtered_df = filtered_df[
                (filtered_df['Lactation'] == lactation) |
                (filtered_df['Lactation'].isna() & (lactation == 'N'))
            ]

        # Check if there are any rows matching the criteria
        if not filtered_df.empty:
            # Convert filtered DataFrame to list of dictionaries
            nutri_data = filtered_df.to_dict(orient='records')

             # Print each dictionary in the desired format
            for record in nutri_data:
                for key, value in record.items():
                    if isinstance(value, float) and math.isnan(value):
                        value = 'null'
                    print(f"{key}: {value}")
        else:
            print("No data found matching the provided criteria.")

        NutritionCalList.append([name, weight, height, bmi, bmi_cat, user_age, gender, pregnancy, lactation, nutri_data])
        print(question)
        name = input().strip().capitalize()
    return NutritionCalList, nutri_data

In [8]:
def load_data(filename='NutritionCalList'):
    NutritionCalList = []
    with open(filename, 'r') as fp:
        NutritionCalList = json.load(fp)
    return NutritionCalList

In [9]:
def store_data(NutritionCalList, filename='NutritionCalList'):
    with open(filename, 'w') as fp:
        json.dump(NutritionCalList, fp)

In [11]:
def main():
    df = pd.read_excel('/content/Estimated Average Requirement Values.xlsx', skiprows=2, skipfooter=3)

    intro="Nutrition Calculator by Yongshi Lin"
    print(intro)
    done = False
    while not done:
        cmd = input(mainmenu)
        if cmd.lower() == 'r':
            NutritionCalList = []
            NutritionCalList, nutri_data = askData(df, NutritionCalList)  # Pass NutritionCalList as a parameter
            printResults(NutritionCalList, nutri_data)  # Pass nutri_data to printResults
        elif cmd.lower() == 'a':
            try:
                NutritionCalList, nutri_data = askData(df, NutritionCalList)
                printResults(NutritionCalList, nutri_data)
            except UnboundLocalError:
                print(nocalc)
        elif cmd.lower() == 's':
            try:
                filename = input('Enter file name. Hit enter for the default file (NutritionCalList)')
                if filename: store_data(NutritionCalList, filename)
                else: store_data(NutritionCalList)
            except UnboundLocalError:
                print(nocalc)
                input("Hit enter to go to the main menu")
            except: pass
        elif cmd.lower() == 'p':
            try:
                printResults(NutritionCalList, nutri_data)
            except UnboundLocalError:
                print(nocalc)
                input("Hit enter to go to the main menu")
            except: pass
        elif cmd.lower() == 'l':
            while True:
                try:
                    filename = input('Enter a file name. Hit enter for the default file (NutritionCalList)')
                    if filename: NutritionCalList = load_data(filename)
                    else: NutritionCalList = load_data()
                    if NutritionCalList:
                        printResults(NutritionCalList, None)
                        break
                except FileNotFoundError:
                    print("Please provide a valid filename")
        elif cmd.lower() == 'q':
            done = True

In [12]:
if __name__ == '__main__':
    main()

Nutrition Calculator by Yongshi Lin

Main Menu:
A)dd to an existing file 
L)oad a file and view data 
P)rint current data 
R)esset and add new data 
S)ave current list to a file
Q)uit:
r
Enter your name to start or q to quit
apple
Enter your weight (in kilograms): 45
Enter your height (in centimeter): 160
Enter your age(in year)25
Enter your gender (M for male/F for female): f
Are you pregnant (Y/N): n
Are you lactating (Y/N): n
Gender: Females
Pregnancy: null
Lactation: null
Age(year): 19-30 y     
Calcium (mg/d): 800.0
CHO (g/d): 100.0
Protein (g/kg/d): 0.66
Vit A (μg/d)a: 500.0
Vit C (mg/d): 60.0
Vit D (μg/d): 10.0
Vit E (mg/d)b: 12.0
Thiamin (mg/d): 0.9
Ribo- flavin (mg/d): 0.9
Niacin (mg/d)c: 11.0
Vit B6 (mg/d): 1.1
Folate (μg/d)d: 320.0
Vit B12 (μg/d): 2.0
Copper (μg/d): 700.0
Iodine (μg/d): 95.0
Iron (mg/d): 8.1
Magnes- ium (mg/d): 255.0
Molyb- denum (μg/d): 34.0
Phos- phorus (mg/d): 580.0
Sele- nium (μg/d): 45.0
Zinc (mg/d): 6.8
Enter your name to start or q to quit
bee
Enter y